#Step 1: Set Up Your Project Folder

In [18]:
# --- STEP 1: CREATE PROJECT DIRECTORY STRUCTURE ---

# Define the name of our main project folder
PROJECT_NAME = "Utensil_Classifier_V2"

# Create all the necessary folders in one go
import os
os.makedirs(f"{PROJECT_NAME}/data/spoon", exist_ok=True)
os.makedirs(f"{PROJECT_NAME}/data/fork", exist_ok=True)
os.makedirs(f"{PROJECT_NAME}/data/others", exist_ok=True) # For "none of them"
os.makedirs(f"{PROJECT_NAME}/models", exist_ok=True)
os.makedirs(f"{PROJECT_NAME}/notebooks", exist_ok=True)

# Move into our main project folder to keep things organized
%cd {PROJECT_NAME}

print(f"✅ Project folder '{PROJECT_NAME}' and all sub-folders created successfully.")
print("\nYour project structure now looks like this:")
# The '!ls -R' command lists all files and folders recursively
!ls -R

/content/Utensil_Classifier_V2/Utensil_Classifier_V2/Utensil_Classifier_V2/Utensil_Classifier_V2/Utensil_Classifier_V2
✅ Project folder 'Utensil_Classifier_V2' and all sub-folders created successfully.

Your project structure now looks like this:
.:
data  models  notebooks

./data:
fork  others  spoon

./data/fork:

./data/others:

./data/spoon:

./models:

./notebooks:


#Step 2: Data Collection & Upload

#Step 3: Create and Run the Training Notebook

In [19]:
# --- STEP 3 (REVISED): CREATE AND POPULATE THE TRAINING NOTEBOOK ---

# This command writes the entire Python script into the notebook file.
# The 'py' at the end of the first line tells Colab to treat this as Python code.
%%writefile notebooks/train_multiclass_model.ipynb
#
# ==============================================================================
# SCRIPT FOR: train_multiclass_model.ipynb
# GOAL: Train a model to classify images into three categories:
#       spoon, fork, and others.
# ==============================================================================

# --- 1. IMPORT LIBRARIES ---
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

print("TensorFlow Version:", tf.__version__)

# --- 2. DEFINE PROJECT PARAMETERS ---
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 8
DATA_DIR = '../data/'
MODEL_SAVE_PATH = '../models/multiclass_detector.h5'

# --- 3. PREPARE THE DATA ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    horizontal_flip=True,
    validation_split=0.2
)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = validation_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

print(f"Class indices found: {train_generator.class_indices}")
num_classes = len(train_generator.class_indices)

# --- 4. BUILD THE MODEL ---
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# --- 5. COMPILE THE MODEL ---
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# --- 6. TRAIN THE MODEL ---
print("\n--- Starting Model Training ---")
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)
print("--- Model Training Complete ---")

# --- 7. SAVE THE FINAL TRAINED MODEL ---
model.save(MODEL_SAVE_PATH)
print(f"\n✅ Model successfully saved to: {MODEL_SAVE_PATH}")

# --- 8. VISUALIZE TRAINING RESULTS ---
acc = history.history['accuracy']
val_acc = history.history['validation_accuracy']
loss = history.history['loss']
val_loss = history.history['validation_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy', marker='o')
plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='o')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid(True)
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss', marker='o')
plt.plot(epochs_range, val_loss, label='Validation Loss', marker='o')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.show()

Writing notebooks/train_multiclass_model.ipynb


#optimized  note book

In [20]:
# --- CREATE THE NEW, ADVANCED TRAINING NOTEBOOK ---

%%writefile notebooks/train_optimized_model.ipynb
#
# ==============================================================================
# NOTEBOOK: train_optimized_model.ipynb
# PURPOSE:  To train a highly accurate model using advanced techniques like
#           Learning Rate Scheduling and Fine-Tuning.
# ==============================================================================

# --- 1. IMPORT LIBRARIES ---
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau # For optimization
import matplotlib.pyplot as plt
import os

print(f"TensorFlow Version: {tf.__version__}")
print(f"Timestamp (Bareilly, India): {os.popen('TZ=\"Asia/Kolkata\" date').read().strip()}")


# --- 2. DEFINE PROJECT PARAMETERS ---
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 8
DATA_DIR = '../data/'
# We will save two models: the initial one, and the final fine-tuned one.
INITIAL_MODEL_SAVE_PATH = '../models/multiclass_detector_initial.h5'
FINAL_MODEL_SAVE_PATH = '../models/multiclass_detector_fine_tuned.h5' # This is our best model


# --- 3. PREPARE DATA GENERATORS ---
# We add more augmentation like brightness adjustments for better generalization.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = validation_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

print(f"Class indices found: {train_generator.class_indices}")
num_classes = len(train_generator.class_indices)


# --- 4. PHASE 1: INITIAL TRAINING (WITH FROZEN BASE) ---
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')
base_model.trainable = False # Keep the pre-trained layers frozen

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define the Learning Rate Scheduler to reduce LR when validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6)

print("\n--- Starting INITIAL Training Phase (20 Epochs) ---")
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)
model.save(INITIAL_MODEL_SAVE_PATH)
print(f"Initial model saved to {INITIAL_MODEL_SAVE_PATH}")


# --- 5. PHASE 2: FINE-TUNING ---
print("\n--- Starting FINE-TUNING Phase (15 More Epochs) ---")

# Unfreeze the top layers of the base model to allow them to be trained
base_model.trainable = True
# We'll fine-tune from the 100th layer onwards. Early layers detect basic shapes and should remain frozen.
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Re-compile the model with a very low learning rate. This is CRITICAL for fine-tuning.
model.compile(
    optimizer=Adam(learning_rate=1e-5), # 0.00001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary() # Print the model summary to see trainable parameters

# Continue training the model to fine-tune the unfrozen layers
fine_tune_epochs = 15
total_epochs = 20 + fine_tune_epochs

history_fine_tune = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1], # Start from where the last training phase ended
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)


# --- 6. SAVE THE FINAL, OPTIMIZED MODEL ---
model.save(FINAL_MODEL_SAVE_PATH)
print(f"\n✅ Final optimized model saved to: {FINAL_MODEL_SAVE_PATH}")

Writing notebooks/train_optimized_model.ipynb
